In [1]:
import cv2
import numpy as np

In [10]:
# connect camera device
cap = cv2.VideoCapture(0)
while cap.isOpened():
    # read frame
    ret, frame = cap.read()
    # reduce video size by half to increase speed
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5,
                       interpolation=cv2.INTER_AREA)
    
    if cv2.waitKey(1) == 27:
        break
    
    # convert to gray scale
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # apply Gaussian blur filter to remove noise(essential before applying Laplacian filter)
    img_gray = cv2.GaussianBlur(img_gray, (9, 9), 0)
    # edge detection using a Laplacian filter
    edges = cv2.Laplacian(img_gray, -1, None, 5)
    # using threshold to keep only boundary value and inverting the screen (white background black lines)
    ret, sketch = cv2.threshold(edges, 10, 255, cv2.THRESH_BINARY_INV)
    
    # dilatation operation to emphasize borderlines
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    sketch = cv2.dilate(sketch, kernel)
    # apply median blur filter to achieve a natural border
    sketch = cv2.medianBlur(sketch, 5)
    # convert from gray scale to BGR color scale
    img_sketch = cv2.cvtColor(sketch, cv2.COLOR_GRAY2BGR)
    # apply average blur filter to remove color image sharpness
    img_paint = cv2.blur(frame, (10, 10))
    
    # combining color image and sketch image
    img_paint = cv2.bitwise_and(img_paint, img_paint, mask=sketch)
    # output result
    merged = np.hstack((img_sketch, img_paint))
    cv2.imshow('Sketch Camera', merged)
    
cap.release()
cv2.destroyAllWindows()